In [1]:
# 导入包
import pandas as pd

C:\Users\ASUS\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\ASUS\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def load_Data(path):
    """加载数据"""
    Data = pd.read_csv(path, nrows=5000)
    Ratings = Data.pivot_table(index='userId', columns='movieId', values='rating') #将数据转换成用户-评分矩阵
    return Ratings

In [3]:
path = r'G:\github项目\Recomendation_system\Data\movie\ratings.csv'
UserItemMatrix = load_Data(path) # 加载数据

In [4]:
UserItemMatrix  # 简单的观察用户-评分矩阵，存在许多空值

movieId,1,2,3,4,5,6,7,8,10,11,...,171023,173307,176101,176371,177593,180095,180777,185135,187541,187593
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,4.0,5.0,3.0,5.0,4.0,4.0,3.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def person_similarity(UserItemMatrix, based='user'):
    """计算皮尔逊相关系数，这里是基于用户的UserCF，若计算ItemCF可改为item"""
    if based == 'user':
        similarity = UserItemMatrix.T.corr(method='pearson') # corr是按列计算相关性，故这里计算用户相关性需要将用户矩阵转置
    else:
        similarity = UserItemMatrix.corr(method='pearson')
    return similarity
UserSimilarity = person_similarity(UserItemMatrix)

In [6]:
UserSimilarity  # 得到各用户的相关系数

userId,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,NaN,0.079819,0.207983,0.268749,-2.916358e-01,-1.187728e-01,4.696682e-01,0.918559,-0.037987,...,-3.643717e-01,0.249046,-0.166667,1.414214e-01,0.189437,-0.014257,-0.070495,-0.142857,3.336630e-02,0.280692
2,NaN,1.000000e+00,NaN,NaN,NaN,NaN,-9.912407e-01,NaN,NaN,0.037796,...,NaN,-0.641689,0.108465,NaN,NaN,-0.166667,-0.310087,NaN,NaN,NaN
3,0.079819,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.204140,NaN,NaN,NaN,NaN
4,0.207983,NaN,NaN,1.000000,-0.336525,1.484982e-01,5.428612e-01,1.178511e-01,NaN,0.485794,...,-2.294804e-01,0.151800,NaN,NaN,0.101143,0.125013,0.455842,NaN,-1.985167e-01,0.469871
5,0.268749,NaN,NaN,-0.336525,1.000000,4.316590e-02,1.581139e-01,2.834734e-02,NaN,-0.777714,...,0.000000e+00,0.193649,NaN,5.867387e-01,0.000000,0.201241,-0.681385,0.500000,1.000000e+00,0.325720
6,-0.291636,NaN,NaN,0.148498,0.043166,1.000000e+00,-1.265950e-01,-2.000621e-01,NaN,0.957427,...,-4.050463e-01,0.077758,NaN,-8.811451e-17,0.482119,-0.138378,0.714577,1.000000,4.338310e-01,-0.204484
7,-0.118773,-9.912407e-01,NaN,0.542861,0.158114,-1.265950e-01,1.000000e+00,2.204155e-01,0.925000,-0.156764,...,1.757808e-01,-0.057351,NaN,-9.950372e-02,-0.117892,0.000068,-0.428521,0.105577,3.590662e-01,-0.151142
8,0.469668,NaN,NaN,0.117851,0.028347,-2.000621e-01,2.204155e-01,1.000000e+00,NaN,-1.000000,...,3.244428e-01,0.166667,1.000000,9.465540e-02,-0.108465,0.056400,-0.298807,NaN,8.164966e-01,0.416025
9,0.918559,NaN,NaN,NaN,NaN,NaN,9.250000e-01,NaN,1.000000,NaN,...,NaN,1.000000,NaN,NaN,1.000000,-0.057640,NaN,NaN,NaN,NaN


In [8]:
def select_similar_user(UserSimilarity, k, uid):
    """根据用户相似度返回与指定用户最相似的前k个用户"""
    UserSimilarity = UserSimilarity.loc[uid].dropna()  # 挑出指定用户，并排除空值
    SimilarUser = []
    for Uid in UserSimilarity.index:
        if UserSimilarity.loc[Uid] != None:
            SimilarUser.append([Uid, UserSimilarity.loc[Uid]])
    SimilarUser.sort(key=lambda x: -x[1]) # 按照相似度降序排序
    SimilarUser = [SimilarUser[i][0] for i in range(len(SimilarUser))]
    return SimilarUser[:k]
select_similar_user(UserSimilarity, 20, 1)

[1, 9, 13, 8, 20, 19, 32, 5, 24, 15, 14, 4, 18, 16, 27, 26, 21, 3, 17, 31]

In [10]:
def user_likemovie(UserItemMatrix, uid):
    """返回指定用户评过分的电影Id"""
    like_movies = []
    movies = UserItemMatrix.loc[uid].dropna()
    for i in movies.index:
        like_movies.append([i, movies.loc[i]]) # 得到指定用户评过分的所有电影
    like_movies.sort(key=lambda x: -x[1])  # 按评分降序排序
    like_movies = [like_movies[i][0] for i in range(len(like_movies))]
    return list(like_movies)
m = user_likemovie(UserItemMatrix, 1)

In [19]:
def recommend_movie(uid, UserItemMatrix, k, UserSimilarity):
    """给指定用户推荐其未看过的电影，根据用户相似度进行推荐"""
    similar_users = select_similar_user(UserSimilarity, k, uid)
    moviesHold = []
    for similar_user in similar_users:
        moviesHold.append(user_likemovie(UserItemMatrix, similar_user)) # 添加待选的电影
    selfs_movies = list(UserItemMatrix.loc[uid].dropna().index) # 选出自己看过的电影
    movies_count = {}
    recommend_movies = []
    for movies in moviesHold:
        for movie in movies:
            if movie not in movies_count:
                movies_count[movie] = 1
            else:
                movies_count[movie] += 1
    for movie in movies_count:
        if movies_count[movie] > k // 3:
            recommend_movies.append(movie) # 若相似的电影推荐数量超过相似用户的1/5，便将其推荐给该用户
    return list(set(recommend_movies) & set(selfs_movies))  # 取推荐电影和自己看过电影的交集
recommend_movie(1, UserItemMatrix, 10, UserSimilarity)

[1,
 260,
 2571,
 780,
 527,
 661,
 296,
 1198,
 47,
 50,
 2617,
 316,
 457,
 592,
 3793,
 593,
 596,
 2012,
 733,
 608,
 356,
 2916,
 367,
 1265,
 1270,
 3578]

In [20]:
def predict_score(uid, iid, UserItemMatrix, similarity):
    """预测给定用户对给定用户的评分"""
    similar_users = similarity.loc[uid].drop([uid]).dropna()  # 排除掉自身
    similar_users = similar_users.where(similar_users > 0.20).dropna()  # 选相似度大于0.2的进行计算
    indexes = set(UserItemMatrix[iid].dropna().index) & set(similar_users.index)
    similar_users = similar_users.loc[list(indexes)]
    
    sum_up = 0
    sum_down = 0
    for sim_uid, similarity in similar_users.iteritems():
        sim_user_similarity_movie = UserItemMatrix.loc[sim_uid]
        sim_user_rating_for_item = sim_user_similarity_movie[iid]
        sum_up += similarity * sim_user_rating_for_item
        sum_down += similarity
    return sum_up / (sum_down + 1e-5)
predict_score(1, 1, UserItemMatrix, UserSimilarity)  # 预测uid为1对iid为1的评分，实际为4

3.4420614989638345

In [22]:
def predict_all(UserItemMatrix, similarity):
    """预测全部用户对物品的评分"""
    user_id = UserItemMatrix.index
    item_id = UserItemMatrix.columns
    Predict_Matrix = pd.DataFrame(index=user_id, columns=item_id) # 新建一个与原先用户评分矩阵相同的空矩阵
    error = 0
    actual_value = 0
    for uid in user_id:
        for iid in item_id:
            Predict_Matrix.loc[uid].loc[iid] = predict_score(uid, iid, UserItemMatrix, similarity)
            if UserItemMatrix.loc[uid].loc[iid] != None:
                error += abs(UserItemMatrix.loc[uid].loc[iid] - Predict_Matrix.loc[uid].loc[iid])  # 按照已有的评分进行准确率评估
                actual_value += UserItemMatrix.loc[uid].loc[iid]
    accuracy = error / actual_value
    return Predict_Matrix, accuracy
predict_all(UserItemMatrix, UserSimilarity)

(movieId    1         2         3         4         5         6         7       \
 userId                                                                          
 1        3.442061  3.330761   2.99995  2.999869       0.0  3.787506  2.946682   
 2             0.0       0.0       0.0       0.0       0.0       0.0       0.0   
 3             0.0       0.0       0.0       0.0       0.0  3.499829       0.0   
 4        3.459042  2.999952  3.306777       0.0       0.0  4.076312  3.999915   
 5        4.381077   2.99989   3.31019       0.0   2.99997  3.851423  3.659671   
 6        3.864142  3.503792  2.999884  2.999892  2.999931  4.999897   3.18007   
 7        4.468822  3.251139       0.0  2.999963  2.999916  3.999806  3.307935   
 8        4.265596   2.99996  3.530243       0.0  2.999963  3.888989  3.999968   
 9        3.836984  3.358378  3.484746       0.0       0.0   4.23026   1.99998   
 10         3.1456  3.928468  4.999948  2.999979  4.999948  3.999958  3.656905   
 11       3.2993